In [1]:
import hydra
import hydra.experimental
import numpy as np
import tqdm
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import RobertaTokenizer
from omegaconf import DictConfig
import math
from torchfly.text.decode import TransformerDecoder
from torchfly.common import set_random_seed, move_to_device
from fast_bleu import BLEU, SelfBLEU
from configure_dataloader import DataLoaderHandler
from model import Generator, TextGAILModel
import os
import logging

from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from transformers import AutoTokenizer
from transformers import RobertaTokenizer, RobertaModel,  RobertaForSequenceClassification

In [2]:
random_seed = 1
set_random_seed(random_seed)

1

In [3]:
hydra.experimental.initialize("./config")

hydra.experimental.initialize()

In [4]:
config = hydra.experimental.compose("config.yaml")

/home/hqh/.conda/envs/torch1.10/lib/python3.9/site-packages/hydra/plugins/config_source.py:190: UserWarning: 
Missing @package directive task/COCO.yaml in file:///home/hqh/Triple-Gan/Evaluation/config.
See https://hydra.cc/docs/next/upgrades/0.11_to_1.0/adding_a_package_directive
  warnings.warn(message=msg, category=UserWarning)
/home/hqh/.conda/envs/torch1.10/lib/python3.9/site-packages/hydra/plugins/config_source.py:190: UserWarning: 
Missing @package directive training/COCO.yaml in file:///home/hqh/Triple-Gan/Evaluation/config.
See https://hydra.cc/docs/next/upgrades/0.11_to_1.0/adding_a_package_directive
  warnings.warn(message=msg, category=UserWarning)
/home/hqh/.conda/envs/torch1.10/lib/python3.9/site-packages/hydra/plugins/config_source.py:190: UserWarning: 
Missing @package directive text_gail/COCO.yaml in file:///home/hqh/Triple-Gan/Evaluation/config.
See https://hydra.cc/docs/next/upgrades/0.11_to_1.0/adding_a_package_directive
  warnings.warn(message=msg, category=UserWarn

In [5]:
# for key in config.keys():
#     config[key]=config[key]
print(config)

{'task': {'name': 'COCO', 'data_dir': '/home/hqh/Triple-Gan/StyleCaption', 'weights_path': 'model_state.pth'}, 'training': {'entropy_lamda': 0, 'mode': 'IPOT', 'random_seed': 1, 'num_gpus_per_node': 1, 'batch_size': 32, 'local_rank': 0, 're_allocate': False, 'triple': True, 'resume': {'resume': False, 'resume_model': True, 'resume_optimizer': True, 'resume_scheduler': True, 'resume_rng_state': True}, 'checkpointing': {'directory': 'Checkpoints', 'steps_interval': 10, 'seconds_interval': -1, 'num_checkpoints_to_keep': 1000, 'keep_checkpoint_every_num_seconds': 86400}, 'logging': {'level': 'INFO', 'steps_interval': -1, 'seconds_interval': 2}, 'optimization': {'fp16': False, 'fp16_opt_level': 'O1', 'optimizer_name': 'AdamW', 'learning_rate': 1e-05, 'gradient_accumulation_steps': 1, 'weight_decay': 0.01, 'max_gradient_norm': -1.0, 'warmup': {'scheduler_name': 'WarmupLinear', 'warmup_steps': 100}}, 'validation': {'steps_interval': 1}, 'total_num': {'epochs': -1, 'update_steps': 500}}, 'text

In [158]:
model_path = '/home/hqh/Triple-Gan/TextGAIL/outputs/2022-06-01/21-09-43/Checkpointsbest_f1_bleu_model.pth'
device = "cuda:0"

In [7]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

In [8]:
model = TextGAILModel(config)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'rob

In [9]:
decoder = TransformerDecoder(config.decode)
decoder.register_generator(model.generator.decoder.to(device))
decoder.register_tokenizer(tokenizer)

In [159]:
weights = torch.load(model_path,map_location = device)
#model.load_state_dict(weights)
model.load_state_dict(weights)

<All keys matched successfully>

In [160]:
device = "cuda:0"
styles =['fact','romantic','funny']
s = {}
s["fact"] = torch.tensor([1,0,0,0])
s["romantic"] = torch.tensor([0, 1, 0,0])
s["funny"] = torch.tensor([0, 0, 1,0])
num = 256

for style in styles:
    f_write = open(style+f"-GEN.txt", "w")
    generated = []
#     for i in range (5):
    model_inputs = {}
    model_inputs["input_ids"] = torch.zeros((num,1),dtype=int).to(device)
#     past = torch.randn(size=(12, 2, num, 12, 1, 61))  # .to(self.device)  # 61=64-3
#     temp = s[style].unsqueeze(0).unsqueeze(0).unsqueeze(0).unsqueeze(0).unsqueeze(0)
#     classification = torch.tile(temp, (12, 2, num, 12, 1, 1))
#     model_inputs['past'] = torch.cat((classification, past), dim=-1).to(device)
    temp = s[style].unsqueeze(0).unsqueeze(0).unsqueeze(0).unsqueeze(0).unsqueeze(0)
    model_inputs['past'] = torch.tile(temp, (12, 2, num, 12, 1, 16)).to(device)
    print(len(model_inputs["input_ids"]))
    #input = move_to_device(torch.zeros((512,1),dtype=int), device)
    results = decoder.generate(model_inputs=model_inputs, temperature=0.6)


    for i in range(len(results["tokens"])):
        res = tokenizer.decode(results["tokens"][i][0][1:-1].tolist())
        generated.append(res)
    for gen in generated:
        f_write.write(json.dumps(gen))
        f_write.write("\n")
    f_write.close()

256
256
256


## 同时可以测试class

In [172]:
device = "cuda:0"
classifier_model = model.classifier.model.to(device)

In [13]:
batch_size = 64

In [14]:
class Seq2SeqDataset(Dataset):
    """
    A Simple Seq2Seq Dataset Implementation
    """
    def __init__(self, fact_filename, romantic_filename,funny_filename, tokenizer, add_bos_token=True, add_eos_token=True):
        data = []
        if fact_filename is not None:
            with open(fact_filename,'r') as f:
                line = f.readline()
                while line:
                    data.append({"source":"","target":line.replace('\n',''),"style":"fact"})
                    line = f.readline()
        if romantic_filename is not None:
            with open(romantic_filename,'r') as f:
                line = f.readline()
                while line:
                    data.append({"source":"","target":line.replace('\n',''),"style":"romantic"})
                    line = f.readline() 
                    
        if funny_filename is not None:
            with open(funny_filename,'r') as f:
                line = f.readline()
                while line:
                    data.append({"source":"","target":line.replace('\n',''),"style":"funny"})
                    line = f.readline()    

        self.data = data
        self.tokenizer = tokenizer
        self.add_bos_token = add_bos_token
        self.add_eos_token = add_eos_token

    def __getitem__(self, index):
        item = self.data[index]
        target_token_ids = self.tokenizer.encode(item["target"], add_special_tokens=False)

        if self.add_bos_token:
            target_token_ids.insert(0, self.tokenizer.bos_token_id)

        if self.add_eos_token:
            target_token_ids.append(self.tokenizer.eos_token_id)


        item["target_token_ids"] = torch.LongTensor(target_token_ids)
        
        if item["style"]=='fact':
            item["source_token_ids"] = [1,0,0]
        elif item["style"]=='romantic':
            item["source_token_ids"] = [0, 1, 0]
        elif item["style"]=='funny':
            item["source_token_ids"] = [0, 0, 1]
        return item

    def __len__(self):
        return len(self.data)

    def collate_fn(self, batch):
        new_batch = {}
        new_batch["source_token_ids"] = torch.tensor([item["source_token_ids"] for item in batch])
        new_batch["target_token_ids"] = pad_sequence(
            [item["target_token_ids"] for item in batch], batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        new_batch["style"] = [item["style"] for item in batch]
        return new_batch



In [15]:
fact_filename = "../StyleCaption/fact-test.txt"
romantic_filename = "../StyleCaption/romantic-test.txt"
funny_filename = "../StyleCaption/funny-test.txt"
test_dataset = Seq2SeqDataset(fact_filename,romantic_filename,funny_filename , tokenizer)
test_dataloader = DataLoader(
            test_dataset, batch_size=batch_size, shuffle=True, collate_fn=test_dataset.collate_fn)

In [16]:
fact_filename = "../StyleCaption/fact-val.txt"
romantic_filename = "../StyleCaption/romantic-val.txt"
funny_filename = "../StyleCaption/funny-val.txt"
valid_dataset = Seq2SeqDataset(fact_filename,romantic_filename,funny_filename , tokenizer)
valid_dataloader = DataLoader(
            valid_dataset, batch_size=batch_size, shuffle=True, collate_fn=valid_dataset.collate_fn)

In [173]:
with torch.no_grad():
    epoch_num = 0 
    epoch_real= 0
    fact_epoch=0
    romantic_epoch = 0
    funny_epoch=0
    for batch in tqdm.tqdm(valid_dataloader):
        sequence = batch['target_token_ids'].to(device)
        label_onehot = batch['source_token_ids'] # n*3,tensor
        label = torch.argmax(label_onehot,dim=-1).to(device) # n tensor
        sequence_logits = classifier_model(sequence).logits
        #打印
        pred_true = torch.argmax(sequence_logits,dim=-1) == label
        batch_real = ( pred_true).sum()
        batch_num = len(label)
        epoch_real += batch_real
        epoch_num += len(label)
        #分门别类
        fact_epoch +=  pred_true[(label==0).nonzero().reshape(-1)].sum()
        romantic_epoch +=  pred_true[(label==1).nonzero().reshape(-1)].sum()
        funny_epoch +=  pred_true[(label==2).nonzero().reshape(-1)].sum()
    acc_rate = epoch_real/epoch_num
    print("在验证集上,正确个数：{},总个数：{},准确率:{},fact:{},romantic:{},funny:{}"
      .format(epoch_real,epoch_num,acc_rate,fact_epoch*3/epoch_num,romantic_epoch*3/epoch_num,funny_epoch*3/epoch_num))
    
    epoch_num = 0 
    epoch_real= 0
    fact_epoch=0
    romantic_epoch = 0
    funny_epoch=0
    for batch in tqdm.tqdm(test_dataloader):
        sequence = batch['target_token_ids'].to(device)
        label_onehot = batch['source_token_ids'] # n*3,tensor
        label = torch.argmax(label_onehot,dim=-1).to(device) # n tensor
        sequence_logits = classifier_model(sequence).logits
        #打印
        pred_true = torch.argmax(sequence_logits,dim=-1) == label
        batch_real = ( pred_true).sum()
        batch_num = len(label)
        epoch_real += batch_real
        epoch_num += len(label)
        #分门别类
        fact_epoch +=  pred_true[(label==0).nonzero().reshape(-1)].sum()
        romantic_epoch +=  pred_true[(label==1).nonzero().reshape(-1)].sum()
        funny_epoch +=  pred_true[(label==2).nonzero().reshape(-1)].sum()
    acc_rate = epoch_real/epoch_num
    print("在测试集上,正确个数：{},总个数：{},准确率:{},fact:{},romantic:{},funny:{}"
      .format(epoch_real,epoch_num,acc_rate,fact_epoch*3/epoch_num,romantic_epoch*3/epoch_num,funny_epoch*3/epoch_num))

100%|██████████| 24/24 [00:00<00:00, 32.95it/s]


在验证集上,正确个数：999,总个数：1500,准确率:0.6660000085830688,fact:0.6980000138282776,romantic:0.6499999761581421,funny:0.6499999761581421


100%|██████████| 24/24 [00:00<00:00, 34.99it/s]

在测试集上,正确个数：992,总个数：1500,准确率:0.6613333225250244,fact:0.7179999947547913,romantic:0.6599999666213989,funny:0.6060000061988831


## 训练好的分类器来评价风格控制准确率

In [18]:
class_pth = "/home/hqh/Triple-Gan/classify.pth"
device ="cpu"
my_classifier_model = torch.load(class_pth, map_location=device)
batch_size = 64

In [19]:
fact_filename = "./funny-GEN.txt"
romantic_filename = "./romantic-GEN.txt"
funny_filename = "./funny-GEN.txt"
gen_dataset = Seq2SeqDataset(fact_filename=fact_filename,romantic_filename=romantic_filename,funny_filename=funny_filename,tokenizer=tokenizer)
gen_dataloader = DataLoader(
            gen_dataset, batch_size=batch_size, shuffle=True, collate_fn=gen_dataset .collate_fn)

In [161]:
device ="cpu"
with torch.no_grad():
    epoch_num = 0 
    epoch_real= 0
    fact_epoch=0
    romantic_epoch = 0
    funny_epoch=0
    for batch in tqdm.tqdm(gen_dataloader):
        sequence = batch['target_token_ids'].to(device)
        label_onehot = batch['source_token_ids'] # n*3,tensor
        label = torch.argmax(label_onehot,dim=-1).to(device) # n tensor
        sequence_logits = my_classifier_model(sequence).logits
        #打印
        pred_true = torch.argmax(sequence_logits,dim=-1) == label
        batch_real = ( pred_true).sum()
        batch_num = len(label)
        epoch_real += batch_real
        epoch_num += len(label)
        #分门别类
        fact_epoch +=  pred_true[(label==0).nonzero().reshape(-1)].sum()
        romantic_epoch +=  pred_true[(label==1).nonzero().reshape(-1)].sum()
        funny_epoch +=  pred_true[(label==2).nonzero().reshape(-1)].sum()
    acc_rate = epoch_real/epoch_num
    print("在验证集上,正确个数：{},总个数：{},准确率:{},fact:{},romantic:{},funny:{}"
      .format(epoch_real,epoch_num,acc_rate,fact_epoch*3/epoch_num,romantic_epoch*3/epoch_num,funny_epoch*3/epoch_num))

100%|██████████| 24/24 [00:18<00:00,  1.30it/s]

在验证集上,正确个数：768,总个数：1536,准确率:0.5,fact:0.701171875,romantic:0.5625,funny:0.236328125


## BLEU vs Self-BLEU

In [162]:
def ref_data():
    data = []
    with open('../StyleCaption/funny-test.txt','r') as f:
        line = f.readline()
        while line:
            data.append(line.replace('\n', '').lower())
            line = f.readline()
    with open('../StyleCaption/fact-test.txt','r') as f:
        line = f.readline()
        while line:
            data.append(line.replace('\n', '').lower())
            line = f.readline()
    
    with open('../StyleCaption/romantic-test.txt','r') as f:
        line = f.readline()
        while line:
            data.append(line.replace('\n', '').lower())
            line = f.readline()
    return data

In [163]:
def gen_data():
    data = []
    with open('./funny-GEN.txt','r') as f:
        line = f.readline()
        while line:
            data.append(line.replace('\n', '').lower().replace("\"",""))
            line = f.readline()
    with open('./fact-GEN.txt','r') as f:
        line = f.readline()
        while line:
            data.append(line.replace('\n', '').lower().replace("\"",""))
            line = f.readline()
    
    with open('./romantic-GEN.txt','r') as f:
        line = f.readline()
        while line:
            data.append(line.replace('\n', '').lower().replace("\"",""))
            line = f.readline()
    return data

In [164]:
ref =ref_data()
gen = gen_data()

In [165]:
ref = [tokenizer.encode(item, add_special_tokens=False) for item in ref]
bleu = BLEU(ref, {'4gram': (0.25, 0.25, 0.25, 0.25)})

In [166]:
gen = [tokenizer.encode(item.lower(), add_special_tokens=False) for item in gen]

In [167]:
bleu_score = np.mean(bleu.get_score(gen)['4gram'])
self_bleu = SelfBLEU(gen, {'4gram': (0.25,0.25,0.25,0.25)})
self_bleu_score = np.mean(self_bleu.get_score()['4gram'])

In [168]:
print(bleu_score)

0.6055083638795318


In [169]:
print(self_bleu_score)

0.8095954778851316


## 计算G-score

In [170]:
g=math.sqrt(bleu_score*acc_rate)

In [171]:
print("准确率：{}，BELU：{}，self-BLEU：{}，G-score：{}".format(acc_rate,bleu_score,self_bleu_score,g))

准确率：0.5，BELU：0.6055083638795318，self-BLEU：0.8095954778851316，G-score：0.550231036515015
